# Conexão API - Brasil - Extração de Dados

Busca os dados da API Brasil para consulta de CNPJs.

In [0]:
#Instalação de Bibliotecas necessárias

import time
import requests
import pandas as pd

In [0]:
# Busca os CNPJs extraídos da API do Bacen
cnpj = spark.sql("""
    SELECT DISTINCT regexp_replace(CNPJ_FUNDO,'[^0-9]', '') AS documento     
    FROM cvm_bronze     
""")

lista_cnpj = [row.documento for row in cnpj.collect()]


dados = []

for cnpj in lista_cnpj:
    time.sleep(1)  # Pausa para evitar limite de requisições

    url = f"https://brasilapi.com.br/api/cnpj/v1/{cnpj}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        dados.append({
            "uf": data.get("uf"),
            "cnpj": data.get("cnpj"),
            "razao_social": data.get("razao_social"),
            "cep": data.get("cep")
        })
    else:
        # Adiciona informação de erro para não perder o CNPJ
        dados.append({
            "uf": None,
            "cnpj": cnpj,
            "cep": None,
            "razao_social": None,
            "erro": f"Status {response.status_code}"
        })

if dados:
    df_pandas = pd.DataFrame(dados)
    df = spark.createDataFrame(df_pandas)
    display(df)
else:
    print("Nenhum dado retornado pela API.")


In [0]:
spark.sql(f'DROP TABLE IF EXISTS brasil_cnpj')
documento.write.saveAsTable('brasil_cnpj')